In [2]:
%%writefile utilities.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utilities.py


### Write YAML file

In [3]:
%%writefile file.yaml
file_type: csv
dataset_name: creditcard_data
file_name: creditcard_data
table_name: creditcard_data
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - LIMIT_BAL
    - EDUCATION
    - MARRIAGE
    - AGE
    - PAY_0
    - PAY_1
    - PAY_2
    - PAY_3
    - PAY_4
    - PAY_5
    - PAY_6
    - BILL_AMT1
    - BILL_AMT2
    - BILL_AMT3
    - BILL_AMT4
    - BILL_AMT5
    - BILL_AMT6
    - PAY_AMT1
    - PAY_AMT2
    - PAY_AMT3
    - PAY_AMT4
    - PAY_AMT5
    - PAY_AMT6
    - default payment next month

Overwriting file.yaml


In [4]:
# Read config file
import utilities as util
config_data = util.read_config_file("file.yaml")

In [7]:
config_data

{'file_type': 'csv',
 'dataset_name': 'creditcard_data',
 'file_name': 'creditcard_data',
 'table_name': 'creditcard_data',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['LIMIT_BAL',
  'EDUCATION',
  'MARRIAGE',
  'AGE',
  'PAY_0',
  'PAY_1',
  'PAY_2',
  'PAY_3',
  'PAY_4',
  'PAY_5',
  'PAY_6',
  'BILL_AMT1',
  'BILL_AMT2',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'BILL_AMT6',
  'PAY_AMT1',
  'PAY_AMT2',
  'PAY_AMT3',
  'PAY_AMT4',
  'PAY_AMT5',
  'PAY_AMT6',
  'default payment next month']}

In [8]:
import pandas as pd

file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

NameError: name 'pd' is not defined

In [12]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['country', 'age']
Following YAML columns are not in the file uploaded ['distance', 'price']


0

In [13]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['city', 'age', 'country'], dtype='object')
columns of YAML are: ['city', 'price', 'distance']


In [14]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['country', 'age']
Following YAML columns are not in the file uploaded ['distance', 'price']
validation failed


In [32]:
pd.read_csv("./test_data.csv")

,city,age,Country
0,Delhi,34,India
1,Lima,30,Peru
2,Istanbul,16,Turkey
3,Riyadh,33,Saudi Arabia


In [15]:
df

,city,age,country
0,Delhi,34,NaN
1,Lima,30,NaN
2,Istanbul,16,NaN
3,Riyadh,33,NaN


In [19]:
### Creating test file for this demo:
testdata = {
    'city' : ['Delhi', 'Lima', 'Istanbul','Riyadh'],
    'age' : [34, 30, 16,33],
    'Country' : ['India','Peru','Turkey','Saudi Arabia']
}
import pandas as pd
df = pd.DataFrame(testdata, columns=['city', 'age','Country'])
df.to_csv("test_data.csv",index=False)

In [20]:
df

,city,age,Country
0,Delhi,34,India
1,Lima,30,Peru
2,Istanbul,16,Turkey
3,Riyadh,33,Saudi Arabia


In [21]:
testdata


{'city': ['Delhi', 'Lima', 'Istanbul', 'Riyadh'],
 'age': [34, 30, 16, 33],
 'Country': ['India', 'Peru', 'Turkey', 'Saudi Arabia']}